In [3]:
import pandas as pd 
import numpy as np
import os
import glob
import folium
import folium.plugins
import datetime
from folium.plugins import HeatMapWithTime
import requests

# Initial analysis

In [11]:
path = "/Users/brialforestier/code/BrialFGD/projectUKcrime/raw_data/2021-05"
filename = "2021-05-avon-and-somerset-street.csv"

In [12]:
full_path = path+"/"+filename
full_path

'/Users/brialforestier/code/BrialFGD/projectUKcrime/raw_data/2021-05/2021-05-avon-and-somerset-street.csv'

In [173]:
df = pd.read_csv(full_path)

In [125]:
df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.494870,51.422276,On or near Conference/Exhibition Centre,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
1,4d223210a924499f387156a1ef04bb09024b62bbe807c6...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.515816,51.408717,On or near Caroline Close,E01014399,Bath and North East Somerset 001A,Burglary,Under investigation,NaN
2,5f7d695d4c96d9c018618bfc72ff0157c9ff31e8452638...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.513343,51.408772,On or near Lincoln Close,E01014399,Bath and North East Somerset 001A,Burglary,Under investigation,NaN
3,940708e1bc177f15670ebb12c8d4c0b2ae5bb89d9ee18f...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.495055,51.422132,On or near Cross Street,E01014399,Bath and North East Somerset 001A,Criminal damage and arson,Under investigation,NaN
4,379ea276b5df46854b4f887d930b5f543499606fe08b29...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.515072,51.419357,On or near Stockwood Hill,E01014399,Bath and North East Somerset 001A,Other theft,Under investigation,NaN


In [19]:
df['Crime ID'].describe()

count                                                 11188
unique                                                11126
top       d007771cd6442b3e9452a4d68ecfc3d68fb793daac14fa...
freq                                                      2
Name: Crime ID, dtype: object

In [20]:
df['Month'].describe()

count       14046
unique          1
top       2021-05
freq        14046
Name: Month, dtype: object

In [18]:
df.count()

Crime ID                 11188
Month                    14046
Reported by              14046
Falls within             14046
Longitude                13147
Latitude                 13147
Location                 14046
LSOA code                13147
LSOA name                13147
Crime type               14046
Last outcome category    11188
Context                      0
dtype: int64

In [21]:
df['Crime type'].describe()

count                            14046
unique                              14
top       Violence and sexual offences
freq                              4692
Name: Crime type, dtype: object

In [23]:
df['Crime type'].value_counts()

Violence and sexual offences    4692
Anti-social behaviour           2858
Public order                    1762
Criminal damage and arson       1157
Vehicle crime                    827
Other theft                      783
Shoplifting                      585
Burglary                         528
Drugs                            208
Other crime                      204
Bicycle theft                    167
Robbery                          128
Theft from the person             81
Possession of weapons             66
Name: Crime type, dtype: int64

In [24]:
df['Last outcome category'].value_counts()

Under investigation                                    8459
Unable to prosecute suspect                            2012
Awaiting court outcome                                  233
Formal action is not in the public interest             106
Further investigation is not in the public interest     105
Investigation complete; no suspect identified            97
Local resolution                                         68
Offender given a caution                                 48
Further action is not in the public interest             35
Action to be taken by another organisation               23
Suspect charged as part of another case                   2
Name: Last outcome category, dtype: int64

In [25]:
df['Context'].value_counts()

Series([], Name: Context, dtype: int64)

# Data pull trials

In [88]:
area_list = [dic['id'].replace('-',' ') for dic in requests.get("https://data.police.uk/api/forces").json()]

In [104]:
area_list[3]

'cheshire'

In [105]:
nbh_list = [dic['name'].replace('-',' ') for dic in requests.get(f"https://data.police.uk/api/{area_list[3]}/neighbourhoods").json()]

In [106]:
nbh_list

['Chester City',
 'Boughton',
 'Garden Quarter',
 'Dodleston and Huntington &amp; Farndon',
 'Great Boughton',
 'Handbridge Park',
 'Hoole',
 'Newton',
 'Lache',
 'Blacon',
 'Saughall and Mollington &amp; Elton',
 'Upton',
 'Chester Villages',
 'Frodsham',
 'Helsby',
 'Kingsley &amp; Gowy',
 'Tarvin and Kelsall',
 'Tattenhall &amp; Malpas',
 'Crewe South',
 'Shavington and Rope',
 'Wistaston and Willaston',
 'Crewe Central',
 'Crewe St Barnabas',
 'Crewe West',
 'Crewe East',
 'Crewe North',
 'Leighton',
 'Haslington',
 'Alsager',
 'Audlem &amp; Wybunbury',
 'Bunbury &amp; Wrenbury',
 'Nantwich North and West',
 'Nantwich South and Stapeley',
 'Odd Rode',
 'Sandbach Elworth &amp; Brereton Rural',
 'Dane Valley',
 'Middlewich',
 'Sandbach Ettiley Heath and Wheelock &amp; Heath and East',
 'Sandbach Town',
 'Congleton East',
 'Gawsworth &amp; Chelford',
 'Congleton West',
 'Macclesfield Hurdsfield',
 'Macclesfield Tytherington',
 'Bollington',
 'Prestbury &amp; Alderley Edge',
 'Disley',

In [31]:
crime_list = [dic['name'] for dic in requests.get("https://data.police.uk/api/crime-categories?").json()][1:-1]

In [32]:
crime_list

['Anti-social behaviour',
 'Bicycle theft',
 'Burglary',
 'Criminal damage and arson',
 'Drugs',
 'Other theft',
 'Possession of weapons',
 'Public order',
 'Robbery',
 'Shoplifting',
 'Theft from the person',
 'Vehicle crime',
 'Violence and sexual offences']

# Locate a user by address exploration

In [156]:
pu = 'stockwood hill,somerset , united kingdom'

In [157]:
response = requests.get(f"https://nominatim.openstreetmap.org/search?q={pu}&format=json").json()[0]

In [158]:
response

{'place_id': 113879897,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 107428434,
 'boundingbox': ['51.418391', '51.4204517', '-2.5177195', '-2.5125093'],
 'lat': '51.4193189',
 'lon': '-2.5151124',
 'display_name': 'Stockwood Hill, Keynsham, Bath and North East Somerset, South West England, England, BS31, United Kingdom',
 'class': 'highway',
 'type': 'unclassified',
 'importance': 0.6}

In [159]:
pu_lat = float(response['lat'])
pu_lon = float(response['lon'])
pu_full_add = response['display_name']

In [142]:
type(pu_lon)

float

In [143]:
type(df['Latitude'][2])

numpy.float64

In [144]:
lat = df['Latitude'].sub(pu_lat).abs().min()+pu_lat
lat

51.4193328

In [145]:
lat_index = df['Latitude'].sub(pu_lat).abs().idxmin()
lat_index

5191

In [97]:
lsoa = df.iloc[lat_index+1]['LSOA name']
lsoa

'Bristol 046C'

In [146]:
from math import sin,cos, atan2, sqrt

def lat_long_distance(lat1, long1, lat2, long2):
    R = 6373.0
    dlon = long2 - long1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return R * c

In [147]:
df.head(1)

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,distance
0,NaN,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.49487,51.422276,On or near Conference/Exhibition Centre,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN,11633.490638


In [126]:
df['distance'] = df.apply(lambda x: lat_long_distance(x['Latitude'], x['Longitude'], lat2=pu_lat, long2=pu_lon),axis=1)

In [130]:
df[['Longitude']].idxmin()[0]

13145

In [175]:
def get_LSOA_city_region(df,long,lat):
    df['distance'] = df[['Longitude','Latitude']].apply(lambda x: hs_distance(x[1], x[0], lat2=lat, long2=long),axis=1)
    indx = df[['distance']].idxmin()[0]    
    lsoa = df.iloc[indx]['LSOA name']
    city = lsoa[:-5]
    region = df.iloc[indx]['Reported by']
    return df, lsoa, city, region

In [176]:
df, lsoa, city, region = get_LSOA_city_region(df, pu_lon, pu_lat)

In [177]:
lsoa

'Bath and North East Somerset 001A'

In [178]:
city

'Bath and North East Somerset'

In [179]:
region

'Avon and Somerset Constabulary'

In [180]:
df_sorted = df.sort_values('distance',ascending = False)
df_sorted.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,distance
11014,92833294b0636caa6880ebb521509de8d621223b2c4848...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.922852,54.361057,On or near Parking Area,E01019397,South Lakeland 002C,Violence and sexual offences,Under investigation,NaN,328.247040
13146,c807a43e1c36f9ec76be58156a183d9fd9dc90913d3080...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.761500,53.875705,On or near Mayflower Grove,E01025547,Wyre 006A,Other theft,Under investigation,NaN,273.642894
8365,27bc5a75a03afc4284730ab3ea40429c67bf47075e6501...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.093778,53.644837,On or near George Street,E01005522,Rochdale 001C,Violence and sexual offences,Under investigation,NaN,249.101114
6391,f63971471c2f316987a5960e6b0610b360e1e6b202f843...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.233560,53.488024,On or near Baptist Street,E01033651,Manchester 054B,Violence and sexual offences,Under investigation,NaN,230.819148
6392,609e438cce493c7a34e6560b1ab721c819ea197b12c795...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-1.201089,53.144023,On or near Bus/Coach Station,E01028284,Mansfield 008F,Violence and sexual offences,Under investigation,NaN,211.578776


In [168]:
!pip install haversine

You should consider upgrading via the '/Users/brialforestier/.pyenv/versions/3.8.6/envs/lewagon/bin/python3.8 -m pip install --upgrade pip' command.


In [174]:
import haversine as hs

def hs_distance(lat1, long1, lat2, long2):
    loc1 = (lat1, long1)
    loc2 = (lat2, long2)
    return hs.haversine(loc1,loc2)

In [170]:
df['hs_distance'] = df.apply(lambda x: hs_distance(x['Latitude'], x['Longitude'], lat2=pu_lat, long2=pu_lon),axis=1)

In [171]:
df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,distance,hs_distance
0,NaN,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.494870,51.422276,On or near Conference/Exhibition Centre,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN,55.374711,1.441624
1,4d223210a924499f387156a1ef04bb09024b62bbe807c6...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.515816,51.408717,On or near Caroline Close,E01014399,Bath and North East Somerset 001A,Burglary,Under investigation,NaN,67.590891,1.179889
2,5f7d695d4c96d9c018618bfc72ff0157c9ff31e8452638...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.513343,51.408772,On or near Lincoln Close,E01014399,Bath and North East Somerset 001A,Burglary,Under investigation,NaN,67.374031,1.179166
3,940708e1bc177f15670ebb12c8d4c0b2ae5bb89d9ee18f...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.495055,51.422132,On or near Cross Street,E01014399,Bath and North East Somerset 001A,Criminal damage and arson,Under investigation,NaN,54.627414,1.425540
4,379ea276b5df46854b4f887d930b5f543499606fe08b29...,2021-05,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.515072,51.419357,On or near Stockwood Hill,E01014399,Bath and North East Somerset 001A,Other theft,Under investigation,NaN,0.264251,0.005079


# Merge all data csv and clean the data

In [4]:
import pandas as pd 
import numpy as np
import os
import glob

In [215]:
file_path = '../raw_data/**/*'

## get all raw data files by months and all police regions

In [224]:
import glob
files_list = glob.glob('../raw_data/**/*street.csv', recursive = True)

In [225]:
li = []

for file in files_list:
    data = pd.read_csv(file, index_col=None, header=0)
    li.append(data)

merged = pd.concat(li, axis=0, ignore_index=True)

In [226]:
#save massive merged df to a csv.
merged.to_csv('merged.csv')

In [ ]:
merged = pd.read_csv('merged.csv')

In [227]:
merged.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,c7000c41002f19263d4adec66b911f1c3f5e7eeb1302a3...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.570572,53.607792,On or near Park/Open Space,E01007418,Barnsley 016A,Other theft,Status update unavailable,NaN
1,b8bc1b6cf423a9431734982fffb11f803cf82140702cc7...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.670108,53.553629,On or near Huddersfield Road,E01007426,Barnsley 027D,Robbery,Investigation complete; no suspect identified,NaN
2,8c69cefea36edafe5fa3f992ccc31d3cfd0c9af9a81429...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.879031,53.943807,On or near Cross End Fold,E01010646,Bradford 001A,Bicycle theft,Investigation complete; no suspect identified,NaN
3,7236a8fb307214df61ca7a9b93bd309c0524f9bee58d89...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.882746,53.933807,On or near Cocking Lane,E01010646,Bradford 001A,Burglary,Investigation complete; no suspect identified,NaN
4,92803cebdfde14ad870899c45c6bee398331f33a053bf1...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.881467,53.944510,On or near Kilners Croft,E01010646,Bradford 001A,Criminal damage and arson,Investigation complete; no suspect identified,NaN


In [235]:
merged.shape

(19300718, 12)

## Cleaning the Merged Data Frame

In [236]:
merged = merged[merged['Longitude'].notna()]

In [237]:
merged.shape

(18919352, 12)

In [238]:
merged = merged[merged['Latitude'].notna()]

In [239]:
merged.shape

(18919352, 12)

In [240]:
merged = merged[merged['Location'].notna()]

In [241]:
merged.shape

(18919352, 12)

In [246]:
merged['LSOA code'].isna().sum()

480390

In [248]:
merged = merged[merged['LSOA code'].notna()]

In [249]:
merged.shape

(18438962, 12)

In [250]:
18919352-480390

18438962

In [247]:
merged['Crime type'].isna().sum()

0

In [244]:
merged['Crime type'].describe

<bound method NDFrame.describe of 0                            Other theft
1                                Robbery
2                          Bicycle theft
3                               Burglary
4              Criminal damage and arson
                        ...             
19300713                   Vehicle crime
19300714       Criminal damage and arson
19300715    Violence and sexual offences
19300716    Violence and sexual offences
19300717    Violence and sexual offences
Name: Crime type, Length: 18919352, dtype: object>

In [251]:
merged['other_id']=merged.apply(lambda x: str(x['Longitude'])+str(x['Latitude'])+x['LSOA code']+x['Crime type'], axis=1)

<ipython-input-251-b9154e67a62f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['other_id']=merged.apply(lambda x: str(x['Longitude'])+str(x['Latitude'])+x['LSOA code']+x['Crime type'], axis=1)


In [252]:
merged.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,other_id
0,c7000c41002f19263d4adec66b911f1c3f5e7eeb1302a3...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.570572,53.607792,On or near Park/Open Space,E01007418,Barnsley 016A,Other theft,Status update unavailable,NaN,-1.57057253.607792E01007418Other theft
1,b8bc1b6cf423a9431734982fffb11f803cf82140702cc7...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.670108,53.553629,On or near Huddersfield Road,E01007426,Barnsley 027D,Robbery,Investigation complete; no suspect identified,NaN,-1.67010853.553629E01007426Robbery
2,8c69cefea36edafe5fa3f992ccc31d3cfd0c9af9a81429...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.879031,53.943807,On or near Cross End Fold,E01010646,Bradford 001A,Bicycle theft,Investigation complete; no suspect identified,NaN,-1.87903153.943807E01010646Bicycle theft
3,7236a8fb307214df61ca7a9b93bd309c0524f9bee58d89...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.882746,53.933807,On or near Cocking Lane,E01010646,Bradford 001A,Burglary,Investigation complete; no suspect identified,NaN,-1.88274653.933807E01010646Burglary
4,92803cebdfde14ad870899c45c6bee398331f33a053bf1...,2020-01,West Yorkshire Police,West Yorkshire Police,-1.881467,53.944510,On or near Kilners Croft,E01010646,Bradford 001A,Criminal damage and arson,Investigation complete; no suspect identified,NaN,-1.88146753.94451E01010646Criminal damage and ...


In [253]:
merged.sort_values(['Reported by','Month','LSOA code'], axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')

<ipython-input-253-18fd7a408ccd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.sort_values(['Reported by','Month','LSOA code'], axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')


In [254]:
cleaned_df = merged.drop_duplicates('other_id', keep='first')

In [6]:
#save cleaned df to a csv.
cleaned_df.to_csv('cleaned_police_data.csv')

NameError: name 'cleaned_df' is not defined

In [7]:
cleaned_df = pd.read_csv('../raw_data/cleaned_police_data.csv')

In [8]:
cleaned_df.shape

(3779090, 14)

In [9]:
cleaned_df['Latitude'] = cleaned_df['Latitude'].astype('float64')

In [10]:
cleaned_df['Longitude'] = cleaned_df['Longitude'].astype('float64')

In [62]:
cleaned_df['LSOA name'].value_counts()

City of London 001F          2306
Westminster 013E             1114
Leeds 111B                   1088
Calderdale 008E               998
Wolverhampton 020F            969
                             ... 
Greenwich 001B                 10
Gateshead 023C                 10
Manchester 013F                 8
Cardiff 017A                    8
Central Bedfordshire 023B       6
Name: LSOA name, Length: 34749, dtype: int64

# Extract LSOA code lat/lon and corresponding Police Region

In [ ]:
locations_df = cleaned_df.groupby(
   ['LSOA code','LSOA name','Reported by'],group_keys=False
).agg(
    {
         'other_id':"count",    # count of 
         'Location':"nunique",    # count of 
         'Longitude': "mean",  # get mean of long in a LSOA
         'Latitude': "mean",  # get mean of lat in a LSOA
        
    }
)

In [49]:
locations_df

other_id  Location  \
LSOA code LSOA name           Reported by                                       
E01000001 City of London 001A British Transport Police            6         1   
                              City of London Police             134        15   
                              Metropolitan Police Service         5         2   
E01000002 City of London 001B British Transport Police            8         1   
                              City of London Police             174        20   
...                                                             ...       ...   
W01001957 Swansea 025G        Gwent Police                        1         1   
                              South Wales Police                109        16   
W01001958 Swansea 025H        Dyfed-Powys Police                  2         1   
                              Gwent Police                        1         1   
                              South Wales Police                123        16   

                                                           Longitude  \
LSOA code LSOA name           Reported by                              
E01000001 City of London 001A British Transport Police     -0.097262   
                              City of London Police        -0.097336   
                              Metropolitan Police Service  -0.097379   
E01000002 City of London 001B British Transport Police     -0.089030   
                              City of London Police        -0.092946   
...                                                              ...   
W01001957 Swansea 025G        Gwent Police                 -3.949578   
                              South Wales Police           -3.946339   
W01001958 Swansea 025H        Dyfed-Powys Police           -3.936054   
                              Gwent Police                 -3.930516   
                              South Wales Police           -3.934711   

                                                            Latitude  
LSOA code LSOA name           Reported by                             
E01000001 City of London 001A British Transport Police     51.520200  
                              City of London Police        51.518251  
                              Metropolitan Police Service  51.521314  
E01000002 City of London 001B British Transport Police     51.518400  
                              City of London Police        51.518353  
...                                                              ...  
W01001957 Swansea 025G        Gwent Police                 51.615686  
                              South Wales Police           51.615874  
W01001958 Swansea 025H        Dyfed-Powys Police           51.618311  
                              Gwent Police                 51.615461  
                              South Wales Police           51.617423  

[52167 rows x 4 columns]

In [56]:
locations_df.shape

(52167, 4)

In [63]:
new_df = locations_df.reset_index()

In [64]:
new_df.head()

,LSOA code,LSOA name,Reported by,other_id,Location,Longitude,Latitude
0,E01000001,City of London 001A,British Transport Police,6,1,-0.097262,51.520200
1,E01000001,City of London 001A,City of London Police,134,15,-0.097336,51.518251
2,E01000001,City of London 001A,Metropolitan Police Service,5,2,-0.097379,51.521314
3,E01000002,City of London 001B,British Transport Police,8,1,-0.089030,51.518400
4,E01000002,City of London 001B,City of London Police,174,20,-0.092946,51.518353


In [98]:
lsao_df = new_df[new_df.groupby(['LSOA name'])['other_id'].transform(max) == new_df['other_id']]

In [99]:
lsao_df.head()

,LSOA code,LSOA name,Reported by,other_id,Location,Longitude,Latitude
1,E01000001,City of London 001A,City of London Police,134,15,-0.097336,51.518251
4,E01000002,City of London 001B,City of London Police,174,20,-0.092946,51.518353
7,E01000003,City of London 001C,City of London Police,41,7,-0.095099,51.521301
10,E01000005,City of London 001E,City of London Police,317,29,-0.075878,51.513443
13,E01000006,Barking and Dagenham 016A,Metropolitan Police Service,55,7,0.087690,51.539150


In [100]:
lsao_df.shape

(34749, 7)

In [66]:
population_df = pd.read_csv('../raw_data/population_data.csv')

/Users/brialforestier/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (27,28,29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [101]:
population_df.head(10)

,LSOA code,population
0,E01011949,"1,954"
1,E01011950,"1,257"
2,E01011951,"1,209"
3,E01011952,"1,740"
4,E01011953,"2,033"
5,E01011954,"2,210"
6,E01011955,"1,361"
7,E01011957,"1,299"
8,E01011959,"2,516"
9,E01011960,"1,303"


In [68]:
population_df.shape

(34753, 98)

In [82]:
population_df = population_df[['LSOA code','population']]

In [76]:
population_df.columns = ['LSOA code','population']

In [83]:
population_df.head()

,LSOA code,population
0,E01011949,"1,954"
1,E01011950,"1,257"
2,E01011951,"1,209"
3,E01011952,"1,740"
4,E01011953,"2,033"


In [102]:
lsao_df = lsao_df.merge(population_df,on='LSOA code')

In [92]:
#save lsoa_df to a csv.
lsao_df.to_csv('lsoa_data.csv')

In [103]:
lsao_df.head()

,LSOA code,LSOA name,Reported by,other_id,Location,Longitude,Latitude,population
0,E01000001,City of London 001A,City of London Police,134,15,-0.097336,51.518251,"1,636"
1,E01000002,City of London 001B,City of London Police,174,20,-0.092946,51.518353,"1,558"
2,E01000003,City of London 001C,City of London Police,41,7,-0.095099,51.521301,"1,786"
3,E01000005,City of London 001E,City of London Police,317,29,-0.075878,51.513443,"1,888"
4,E01000006,Barking and Dagenham 016A,Metropolitan Police Service,55,7,0.087690,51.539150,"2,094"


In [104]:
lsao_df.shape

(34749, 8)

In [105]:
lsao_df = lsao_df[['LSOA code','LSOA name','Reported by','Longitude','Latitude','population']]

In [106]:
lsao_df.columns = ['lsoa','lsoa_name','force','lon','lat','pop']

In [107]:
lsao_df.to_pickle('../raw_data/lsoa_data.pkl')

In [108]:
lsao_df.head()

,lsoa,lsoa_name,force,lon,lat,pop
0,E01000001,City of London 001A,City of London Police,-0.097336,51.518251,"1,636"
1,E01000002,City of London 001B,City of London Police,-0.092946,51.518353,"1,558"
2,E01000003,City of London 001C,City of London Police,-0.095099,51.521301,"1,786"
3,E01000005,City of London 001E,City of London Police,-0.075878,51.513443,"1,888"
4,E01000006,Barking and Dagenham 016A,Metropolitan Police Service,0.087690,51.539150,"2,094"


In [109]:
lsao_df['city']=lsao_df['lsoa_name'].apply(lambda x: x[:-5])

In [110]:
lsao_df.head()

,lsoa,lsoa_name,force,lon,lat,pop,city
0,E01000001,City of London 001A,City of London Police,-0.097336,51.518251,"1,636",City of London
1,E01000002,City of London 001B,City of London Police,-0.092946,51.518353,"1,558",City of London
2,E01000003,City of London 001C,City of London Police,-0.095099,51.521301,"1,786",City of London
3,E01000005,City of London 001E,City of London Police,-0.075878,51.513443,"1,888",City of London
4,E01000006,Barking and Dagenham 016A,Metropolitan Police Service,0.087690,51.539150,"2,094",Barking and Dagenham


In [111]:
lsao_df.to_pickle('../raw_data/lsoa_data.pkl')